# EmpkinS Micro – ECG Data Preprocessing

## Imports and Helper Functions

In [ ]:
import json
import sys
sys.path.append("../../scripts")

from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.io import write_pandas_dict_excel
from biopsykit.io.nilspod import load_dataset_nilspod
from biopsykit.signals.ecg import EcgProcessor

from tqdm.notebook import tqdm

from time_logs import load_split_time_logs

import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

## Variable and Path Setup

In [ ]:
config_file = Path("../../config.json")
with open(config_file) as f:
    config_json = json.load(f)
    base_path = Path(config_json["base_path"])
    
base_path = Path("../..").joinpath(base_path)
data_path = base_path.joinpath("data_per_subject")

In [ ]:
subject_dirs = bp.utils.file_handling.get_subject_dirs(data_path, "VP_*")
subject_dirs

## Process ECG Data

In [ ]:
subject_dict = {}

for subject_dir in tqdm(subject_dirs[:1], desc="Subject"):
    subject_id = subject_dir.name
    
    for condition in tqdm(["control", "stress"], desc=subject_id):        
        ecg_dir = subject_dir.joinpath("{}/ecg").format(condition)
        timelog_dir = subject_dir.joinpath("{}/time_log").format(condition)
        timelog_dict = load_split_time_logs(timelog_dir.joinpath("processed"))
        
        ecg_dict, fs = load_folder_nilspod(ecg_dir.joinpath("raw"), phase_names=["Pre", "MIST", "Post"])
        
        ep_dict = {}
        for key_timelog, key_ecg in tqdm(list(zip(timelog_dict, ecg_dict)), desc=condition):
            ep = EcgProcessor(data=ecg_dict[key_ecg], sampling_rate=fs, time_intervals=timelog_dict[key_timelog])
            ep.ecg_process(title=key_ecg)
            
            ep_dict[key_ecg] = ep

            ecg_export_dir = ecg_dir.joinpath("processed")
            ecg_export_dir.mkdir(exist_ok=True)
            write_pandas_dict_excel(ep.heart_rate, ecg_export_dir.joinpath("heart_rate_result_{}_{}.xlsx".format(subject_id, key_timelog)))
            write_pandas_dict_excel(ep.rpeaks, ecg_export_dir.joinpath("rpeaks_result_{}_{}.xlsx".format(subject_id, key_timelog)))
            
        subject_dict[subject_id] = ep_dict
